In [3]:
import sqlite3
import pandas as pd

# 1. 准备一些示例数据（实际项目里可以来自 CSV/接口等）
prices = pd.DataFrame(
    {
        "symbol": ["AAPL", "AAPL", "MSFT", "MSFT"],
        "date": pd.to_datetime(
            ["2024-01-02", "2024-01-03", "2024-01-02", "2024-01-03"]
        ),
        "close": [185.1, 187.3, 330.2, 333.8],
        "volume": [100_000, 120_000, 90_000, 110_000],
    }
)

# 2. 连接到 SQLite 数据库文件（不存在会自动创建）
con = sqlite3.connect("data/data.sqlite")

# 3. 将 pandas DataFrame 写入数据库（存）
prices.to_sql("prices", con, if_exists="replace", index=False)

# 4. 追加新数据（可选）
new_rows = pd.DataFrame(
    {
        "symbol": ["AAPL"],
        "date": pd.to_datetime(["2024-01-04"]),
        "close": [189.0],
        "volume": [130_000],
    }
)
new_rows.to_sql("prices", con, if_exists="append", index=False)

# 5. 查询（取）
result_df = pd.read_sql_query(
    "SELECT symbol, date, close FROM prices WHERE symbol = 'AAPL' ORDER BY date",
    con,
)
print(result_df)

# 6. 定期清理
con.execute("VACUUM")
con.execute("ANALYZE")
con.close()


  symbol       date  close
0   AAPL 2024-01-02  185.1
1   AAPL 2024-01-03  187.3
2   AAPL 2024-01-04  189.0
  symbol       date  close  volume
0   AAPL 2024-01-03  187.3  120000
1   MSFT 2024-01-03  333.8  110000
2   AAPL 2024-01-04  189.0  130000
